# Examen Final
## Curso: Data Mining Tools
## Nombre: Ibrahim Imanol Jordi Arquinigo Jacinto
## Codigo: U20191e650



# Librerias

In [59]:
%pip install deap optuna kagglehub streamlit scikit-learn

In [60]:
# Manipulación de datos
import pandas as pd
import numpy as np
import os

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento y modelos
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
# Para trabajar con fechas y horas
from datetime import datetime
# Modelizacion y optimizacion
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from deap import base, creator, tools, algorithms
import random

import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import optuna
import streamlit as st

# Cargar Datos

In [61]:
import kagglehub
import pandas as pd

# Descargar el dataset desde Kaggle
dataset_path = kagglehub.dataset_download("brendan45774/test-file")

# Ubicar el archivo CSV en la carpeta descargada
csv_file = f"{dataset_path}/tested.csv"  # Ajustar según el nombre real del archivo

# Cargar el dataset en un DataFrame de pandas
data = pd.read_csv(csv_file)

# Mostrar las primeras filas del dataset
print(data.head())


   PassengerId  Survived  Pclass  \
0          892         0       3   
1          893         1       3   
2          894         0       2   
3          895         0       3   
4          896         1       3   

                                           Name     Sex   Age  SibSp  Parch  \
0                              Kelly, Mr. James    male  34.5      0      0   
1              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1      0   
2                     Myles, Mr. Thomas Francis    male  62.0      0      0   
3                              Wirz, Mr. Albert    male  27.0      0      0   
4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1      1   

    Ticket     Fare Cabin Embarked  
0   330911   7.8292   NaN        Q  
1   363272   7.0000   NaN        S  
2   240276   9.6875   NaN        Q  
3   315154   8.6625   NaN        S  
4  3101298  12.2875   NaN        S  


# Descripcion de los datos

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [63]:
# Verificar la cantidad de valores nulos por columna
print("\nValores nulos por columna:")
print(data.isnull().sum())


Valores nulos por columna:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [64]:
# Verificar si hay celdas con cadenas vacías ("")
print("\nCeldas con cadenas vacías:")
print((data == '').sum())


Celdas con cadenas vacías:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [65]:
# Verificar si hay filas duplicadas
print("\nNúmero de filas duplicadas:")
print(data.duplicated().sum())


Número de filas duplicadas:
0


In [66]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,0.363636,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.481622,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,0.000000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,0.000000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,1.000000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,1.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [67]:
data.describe(include=['object'])

,Name,Sex,Ticket,Cabin,Embarked
count,418,418,418,91,418
unique,418,2,363,76,3
top,"Kelly, Mr. James",male,PC 17608,B57 B59 B63 B66,S
freq,1,266,5,3,270


# Preprocesamiento

In [68]:
# Seleccionar solo las columnas relevantes
data = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']]

# Tratar valores nulos

# Llenar los valores nulos en 'Age' con la mediana
data['Age'] = data['Age'].fillna(data['Age'].median())

# Llenar los valores nulos en 'Fare' con la mediana
data['Fare'] = data['Fare'].fillna(data['Fare'].median())

# Llenar los valores nulos en 'Embarked' con el valor más frecuente
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

# Verificar nuevamente los valores nulos
print("Valores nulos después del preprocesamiento:")
print(data.isnull().sum())


# Codificar variables categóricas
label_enc = LabelEncoder()
data['Sex'] = label_enc.fit_transform(data['Sex'])
data['Embarked'] = label_enc.fit_transform(data['Embarked'])

# Separar características (X) y variable objetivo (y)
X = data.drop('Survived', axis=1)
y = data['Survived']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos para modelos sensibles a magnitudes
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Valores nulos después del preprocesamiento:
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
Survived    0
dtype: int64


<ipython-input-68-4132c684c138>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Age'] = data['Age'].fillna(data['Age'].median())
<ipython-input-68-4132c684c138>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Fare'] = data['Fare'].fillna(data['Fare'].median())
<ipython-input-68-4132c684c138>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [69]:
# Función objetivo para Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Función objetivo para Logistic Regression
def objective_lr(trial):
    C = trial.suggest_loguniform('C', 1e-4, 1e2)  # Regularización
    solver = trial.suggest_categorical('solver', ['lbfgs', 'liblinear'])
    max_iter = trial.suggest_int('max_iter', 100, 500)

    model = LogisticRegression(C=C, solver=solver, max_iter=max_iter, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Optimizar Random Forest
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=30)
best_params_rf = study_rf.best_params
print("Mejores hiperparámetros para Random Forest:", best_params_rf)

# Optimizar Logistic Regression
study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=30)
best_params_lr = study_lr.best_params
print("Mejores hiperparámetros para Logistic Regression:", best_params_lr)

# Entrenar modelos optimizados
optimized_rf = RandomForestClassifier(**best_params_rf, random_state=42)
optimized_rf.fit(X_train, y_train)

optimized_lr = LogisticRegression(**best_params_lr, random_state=42)
optimized_lr.fit(X_train, y_train)

# Evaluar los modelos
y_pred_rf = optimized_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Exactitud del modelo Random Forest optimizado: {accuracy_rf}")

y_pred_lr = optimized_lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Exactitud del modelo Logistic Regression optimizado: {accuracy_lr}")



[I 2024-11-26 20:48:50,867] A new study created in memory with name: no-name-309f7d5a-af0b-443c-826f-debd7cc045f8
[I 2024-11-26 20:48:51,287] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 76, 'max_depth': 5, 'min_samples_split': 6, 'min_samples_leaf': 5}. Best is trial 0 with value: 1.0.
[I 2024-11-26 20:48:51,454] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 40, 'max_depth': 12, 'min_samples_split': 4, 'min_samples_leaf': 5}. Best is trial 0 with value: 1.0.
[I 2024-11-26 20:48:51,614] Trial 2 finished with value: 1.0 and parameters: {'n_estimators': 41, 'max_depth': 7, 'min_samples_split': 9, 'min_samples_leaf': 1}. Best is trial 0 with value: 1.0.
[I 2024-11-26 20:48:52,232] Trial 3 finished with value: 1.0 and parameters: {'n_estimators': 102, 'max_depth': 12, 'min_samples_split': 6, 'min_samples_leaf': 6}. Best is trial 0 with value: 1.0.
[I 2024-11-26 20:48:52,856] Trial 4 finished with value: 1.0 and parameters: {'n_estimators': 93, 'max_

Mejores hiperparámetros para Random Forest: {'n_estimators': 76, 'max_depth': 5, 'min_samples_split': 6, 'min_samples_leaf': 5}


[I 2024-11-26 20:49:05,038] Trial 10 finished with value: 1.0 and parameters: {'C': 0.569489343675546, 'solver': 'lbfgs', 'max_iter': 125}. Best is trial 0 with value: 1.0.
<ipython-input-69-cfeabe2aa6af>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)  # Regularización
[I 2024-11-26 20:49:05,074] Trial 11 finished with value: 1.0 and parameters: {'C': 1.1918433644791695, 'solver': 'liblinear', 'max_iter': 485}. Best is trial 0 with value: 1.0.
<ipython-input-69-cfeabe2aa6af>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)  # Regularización
[I 2024-11-26 20:49:05,110] Trial 12 

Mejores hiperparámetros para Logistic Regression: {'C': 33.43809769729899, 'solver': 'lbfgs', 'max_iter': 165}
Exactitud del modelo Random Forest optimizado: 1.0
Exactitud del modelo Logistic Regression optimizado: 1.0


In [70]:
import pickle

# Guardar Random Forest optimizado
with open('optimized_rf.pkl', 'wb') as rf_file:
    pickle.dump(optimized_rf, rf_file)

# Guardar Logistic Regression optimizado
with open('optimized_lr.pkl', 'wb') as lr_file:
    pickle.dump(optimized_lr, lr_file)
